Init



In [0]:
!wget https://github.com/OrtenburgerAdrian/Thesis-Data/archive/master.zip
!unzip master.zip

import re
import string
from google.colab import files

import numpy as np
import os
import subprocess
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras import Sequential
from keras.preprocessing import sequence
from keras.models import load_model
from keras.layers import Embedding, LSTM, Dense, Dropout


--2019-03-20 09:58:04--  https://github.com/OrtenburgerAdrian/Thesis-Data/archive/master.zip
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/OrtenburgerAdrian/Thesis-Data/zip/master [following]
--2019-03-20 09:58:05--  https://codeload.github.com/OrtenburgerAdrian/Thesis-Data/zip/master
Resolving codeload.github.com (codeload.github.com)... 192.30.255.120, 192.30.255.121
Connecting to codeload.github.com (codeload.github.com)|192.30.255.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [   <=>              ]   2.97M  4.89MB/s    in 0.6s    

2019-03-20 09:58:06 (4.89 MB/s) - ‘master.zip’ saved [3115145]

Archive:  master.zip
91a5ba15669ffef0dd4592887ff08fa473422427
   creating: Thesis-Data-master/
  infl

Using TensorFlow backend.


In [0]:
!ls ./Thesis-Data-master/

In [0]:
files.download('6k_spellcheck_word-rechtschreib-prüfung.formt_2.0')

Doppelte Zeilen Löschen

In [0]:
string = ''
fobj = open("Thesis-Data-master/a_c_k")
lines = fobj.readlines()
lines = sorted(lines)
fobj_out = open("a_c_k","w")
for line in lines:
    if string == line.rstrip():
        print(line.rstrip())
    else:
        fobj_out.write(line)
    string = line.rstrip()
fobj.close()
fobj_out.close()


ID-Tabelle ertellen

In [0]:
fobj = open("Thesis-Data-master/a_w_k")
list_w=[]
list_z=[]

for line in fobj:
  line = re.sub('\n' , '', line)
  if line in list_w:
    list_z[list_w.index(line)] += 1 
  else:
    list_w.append(line)
    list_z.append(1)
    

fobj_out = open("c_w_t","w")


for x in list_w:
  if list_z[list_w.index(x)]<3:
    i=list_w.index(x)
    list_w[i]=''
    list_z[i]=0
  
i=0   
for x in list_w:
  if x=='':
    continue
  fobj_out.write(str(x)+' '+str(i)+' '+str(list_z[list_w.index(x)])+'\n')
  i+=1
fobj_out.close()
fobj.close() 

Kommentare mit ID-Tabelle übersätzen

In [0]:
fobj_out = open("c_i_z1","w")
fobj2 = open("Thesis-Data-master/c_w_t")
fobj = open("Thesis-Data-master/a_c_k")
list_w=[]
for w in fobj2:
  list_w.append(w.split(' ')[0])
for line in fobj:
  worter = line.split(';')[1].split(' ')
  x=''
  for wort in worter:
    wort = re.sub('\n' , '', wort)
    wort = wort.lower()
    if wort in list_w:
      if list_w.index(wort)==0:
        index = 3136
      else:
        index = list_w.index(wort)
      x=x+str(index)+' '
  fobj_out.write(str(line.split(';')[0])+';'+ x+'\n')
  
string = ''
fobj = open("c_i_z1")
lines = fobj.readlines()
lines = sorted(lines)
fobj_out = open("c_i_z","w")
for line in lines:
    if string == line.rstrip():
        print(line.rstrip())
    else:
        fobj_out.write(line)
    string = line.rstrip()
fobj.close()
fobj_out.close()

Wie viele wörter bis 90% der Kommentare nicht gekürzt werden müssen.

In [0]:
prozent = 90 # so viele Prozent wie gewüscht 

fobj = open("Thesis-Data-master/c_i_z")
a = []
i = 0
while i < 1000:
  a.append(0)
  i += 1
for x in fobj:
  a[len( x.split(';')[1].split(' '))]+=1
fobj.close()
i = 0
for x in a:
  i = i + x 
y = 0
u = 0
while u <len(a):
  y = y + a[u]
  if y / i * 100 > prozent:
    print (u)
    break
  u += 1

34


Den Sentimentwert in ein Array übertragen

In [0]:
fobj = open("Thesis-Data-master/c_i_z")
lines = fobj.readlines()
Y=[]
for line in lines:
  if line.split(';')[1] == "\n":
    continue
  if line.split(';')[0] == "0":
    Y.extend([[1, 0]])
  if line.split(';')[0] == "1":
    #Y.extend([[0, 1]])
    continue
  if line.split(';')[0] == "2":
    Y.extend([[0, 1]])

fobj.close()
y = np.array(Y, dtype=float)
print(y.shape)

(2485, 2)


Den Kommmentar (die Zahlenkolonne) in ein Array übertragen

In [0]:
fobj = open("Thesis-Data-master/c_i_z")
list_c=[]
for line in fobj:
  list_w=[]
  line = re.sub('\n' , '', line)
  if line.split(';')[1] == "":
    continue  
  if line.split(';')[0] == "1":
    continue
  words = line.split(';')[1].split(' ')
  for word in words:
    try:
      if int(word) == 0: 
        word= 1316
      list_w.append(int(word))
    except:
      continue
  list_c.append(list_w)
print (len(list_c))

2485


Daten in Test-, Trainings-, und Validirungsdaten aufteilen.

In [0]:
max_words = 34
validierungsdaten=50

#aufteilen in Test- und Trainingsdaten
X_train,X_test,y_train,y_test=train_test_split(
    list_c, y,test_size=0.10, random_state = 1)

#Kürzung bzw. Erweiterung der Kommentare auf die maximale Länge.
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

#Aufteilung in Trainings- und Validierungsdaten
X_valid, y_valid = X_train[:validierungsdaten], y_train[:validierungsdaten]
X_train, y_train = X_train[validierungsdaten:], y_train[validierungsdaten:]

Das LSTM

In [0]:
embedding_size=32
library=3137
max_words= 34

model=Sequential()
model.add(Embedding(library, embedding_size, input_length=max_words))
model.add(LSTM(2))
model.add(Dropout(0.25))
#model.add(Dense(2, activation='sigmoid'))
model.add(Dense(2, activation='softmax'))
print(model.summary())
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 34, 32)            100384    
_________________________________________________________________
lstm_3 (LSTM)                (None, 4)                 592       
_________________________________________________________________
dropout_3 (Dropout)          (None, 4)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 10        
Total params: 100,986
Trainable params: 100,986
Non-trainable params: 0
_________________________________________________________________
None


Training

In [0]:
i=1
while i<200:
  model.fit(X_train, y_train, validation_data=(X_valid, y_valid), 
                                          batch_size=1, epochs=1)
  scores = model.evaluate(X_test, y_test, verbose=0)
  print('Test accuracy:', scores[1])
  model.save('modell_acc.'+str(scores[1])+'_epoch.'+str(i))
  i+=1

In [0]:
!ls

a_c_k
c_i_z
c_i_z1
master.zip
model_ger_comm_acc.0.7991967878667227_epoch.3
model_ger_comm_acc.0.8112449789621744_epoch.1
model_ger_comm_acc.0.8112449806378069_epoch.4
model_ger_comm_acc.0.8152610465704676_epoch.6
model_ger_comm_acc.0.8192771074762306_epoch.7
model_ger_comm_acc.0.8232931717332587_epoch.5
model_ger_comm_acc.0.8232931734088913_epoch.9
model_ger_comm_acc.0.8232931738876434_epoch.2
model_ger_comm_acc.0.8473895611054447_epoch.8
sample_data
Thesis-Data-master


In [0]:
files.download('XXXXX')

In [0]:
model = load_model('Thesis-Data-master/model_ger_comm_acc.0.8634538181335571_epoch.12')

In [0]:

results = model.predict(X_test)
i=0
false_negativ=0
true_negativ=0
false_positv=0
true_positv=0

while i< len(results):
  if y_test[i][0] == 0:
    if results[i][0] > 0.5:
      false_positv += 1
    else:
      true_positv += 1
  if y_test[i][0] == 1:
    if results[i][0] > 0.5:
      true_negativ += 1
    else:
      false_negativ += 1
  i+=1
print("false_negativ")
print(false_negativ)
print("true_negativ")
print(true_negativ)
print("false_positv")
print(false_positv)
print("true_positv")
print(true_positv)

false_negativ
16
true_negativ
131
false_positv
18
true_positv
84


Rückübersätzer

In [0]:
results = model.predict(X_test)
fobj2 = open("Thesis-Data-master/c_w_t")
fobj = open("Thesis-Data-master/c_i_z")
list_w=[]

for w in fobj2:
  list_w.append(w.split(' ')[0])


i=0  
print('Sentiment; Vorhergesagt: x > 0.5 == positiv; Kommentar')
print('0=negativ; 1=positiv')
while i < len(X_test):
  com = X_test[i]
  comment = '' 
  #comment = str(int(y_test[i][1]))+';'+str(float(int(results[i][1]*10000))/10000)+';'
  comment = str(int(y_test[i][1]))+';'+"%f"%results[i][1]+';'
  for word in com:
    if word == 0:
      continue
    if word == 3136:
      word=0
    comment = comment+ ' ' + list_w[word]
  comment = re.sub('; ' , ';', comment)
  print(comment)
  i+=1

Sentiment; Vorhergesagt: x > 0.5 == positiv; Kommentar
0=negativ; 1=positiv
0;0.004471;so siehts aus
0;0.009597;einfach nicht mehr bahn fahren
0;0.000178;woanders rum
1;0.998720;habe den gutschein bekommen danke für die info
0;0.000071;schade leider nicht für dich
1;0.998924;richtig
0;0.002172;nimm den und lass ihn ruhig am fenster schauen und einer kümmert sich um den hund allerdings wurde ich den in ein anderes zimmer bringen damit der kleine nicht das selbe verhalten
0;0.000871;lidl und liefern lassen von und die liefern ja aber dann an solche wo es nicht hin soll und der nicht wohnt gerade vor wochen gehabt ein kunde weniger und
0;0.000866;den hauptbahnhof zeigen sie hier nicht weil er einen ist für deutschland ist
1;0.999770;xd xd xd
1;0.024065;ich kann es mir ja kaum vorstellen weil wer macht schon pommes mit einem aber selbst auf youtube die sagen das die dinger was können
1;0.999837;das beste katzenstreu eher
1;0.999872;ich kaufe das katzen streu von anfang an in lidl ich bin s